In [1]:
import pandas as pd
import numpy as np
import re
import string
import requests
import io
import sys
#Antes de hacer esto necesitamos importar el archivo de funciones que se encuentra en el repositorio.
sys.path.append('../')
import funciones as fn

In [2]:
username = 'diegobermu7'
token = 'f27e201806532f825d8327ca2ea52a34d44fb60e'
github_session = requests.Session()
github_session.auth = (username, token)

In [3]:
#'/Users/diegoalejandrobermudezsierra/Documents/BX/Base/Q_Barometro_Migrante_Sep_2020_Feb_2021.xlsx'

# Definimos la ubicación del archivo
url = "https://github.com/terrok9/BX/blob/main/Base/Q_Barometro_Migrante_Sep_2020_Feb_2021.xlsx?raw=true" 
df = pd.read_excel(url)
#Importamos la base de datos que descargamos con los tweets de brandwatch 


df.head(11)

,Report:,Bulk Mentions Download,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,...,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113
0,Brand:,[Q [ Barometro Migrante Colombia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,From:,Wed Sep 30 00:00:00 UTC 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,To:,Tue Feb 02 00:00:00 UTC 2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Label:,Q Barómetro Migrante Sep 30 2020 - Feb 1 2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Query Id,Query Name,Date,Title,Snippet,Url,Domain,Sentiment,Page Type,Language,Country Code,Continent Code,Continent,Country,City Code,Account Type,Added,Assignment,Author,Avatar,Category Details,Checked,City,Display URLs,Expanded URLs,Facebook Author ID,Facebook Comments,Facebook Likes,Facebook Role,Facebook Shares,Facebook Subtype,Full Name,Full Text,Gender,Hashtags,Impact,Impressions,Instagram Comments,Instagram Followers,Instagram Following,...,Twitter Tweets,Twitter Verified,Updated,Reach (new),Blog Name,Categorías prueba - Denuncia,Categorías prueba - Xenofobia,Copyright,Item Review,Linkedin Comments,Linkedin Engagement,Linkedin Impressions,Linkedin Likes,Linkedin Shares,Linkedin Sponsored,Linkedin Video Views,Medios Colombianos por Ciudad - Medios Barranq...,Medios Colombianos por Ciudad - Medios Bogota,Medios Colombianos por Ciudad - Medios Cali,Medios Colombianos por Ciudad - Medios Cucuta,Medios Colombianos por Ciudad - Medios Medellin,Medios Colombianos por Ciudad - Medios Nacionales,Medios Policia Captura - Capturas a Venezolanos,Medios Policia Captura - Todas las capturas,Medios de Comunicación - Alternativos General,Medios de Comunicación - Alternativos Venezolanos,Medios de Comunicación - Tradicionales General,Medios de Comuni

In [ ]:
#Esto lo hacemos para tener una copia de seguridad porque la base se demora 1000000 años cargando 
copia=df.copy()

In [8]:
df=copia.copy()

In [9]:
df=fn.limpieza_base(df)
drop_vars= ["Query_Name", "Url", "Page_Type", "Language", "Country_Code", "Continent_Code", 
            "Country", "Continent", "Added", "Assignment", "Avatar", "Category_Details", 
            "Checked", "Display_URLs", "Expanded_URLs", "Facebook_Author_ID", 
            "Facebook_Comments", "Facebook_Likes", "Facebook_Role", "Facebook_Shares", 
            "Facebook_Subtype", "Full_Text", "Full_Name", "Instagram_Comments", 
            "Instagram_Followers", "Instagram_Following", "Instagram_Likes", 
            "Instagram_Interactions_Count", "Instagram_Posts", "Last_Assignment_Date", 
            "Latitude", "Longitude", "Location_Name", "Media_Filter", "Media_URLs", 
            "Original_Url", "Priority", "Short_URLs", "Resource_Id", "Starred", 
            "Status", "Subtype", "Tags", "Thread_Created_Date", "Thread_Id", "Thread_URL", 
            "Twitter_Author_ID", "Twitter_Channel_Role", "Twitter_Reply_to", 
            "Twitter_Retweet_of", "Updated", "Blog_Name", "Copyright", "Item_Review", 
            "Page_Type_Name", "Linkedin_Comments", "Linkedin_Engagement", 
            "Linkedin_Impressions", "Linkedin_Likes", "Linkedin_Shares", 
            "Linkedin_Sponsored", "Linkedin_Video_Views", "Parent_Blog_Name", 
            "Parent_Post_Id", "Pub_Type", "Publisher_Sub_Type", "Rating", "Reddit_Score", 
            "Region", "Region_Code", "Root_Blog_Name", "Root_Post_Id", "Weblog_Title"]

df=df.drop(drop_vars, axis=1)

df.head()

Escriba la fila donde se encuentran los nombres de las columnas - 5


,Date,Title,Snippet,Domain,Sentiment,City_Code,Account_Type,Author,City,Gender,Hashtags,Impact,Impressions,Interest,Mentioned_Authors,Professions,Thread_Author,Thread_Entry_Type,Total_Monthly_Visitors,Twitter_Followers,Twitter_Following,Twitter_Reply_Count,Twitter_Retweets,Twitter_Tweets,Twitter_Verified,Reach_(new),Categorías_prueba_-_Denuncia,Categorías_prueba_-_Xenofobia,Medios_Colombianos_por_Ciudad_-_Medios_Barranquilla,Medios_Colombianos_por_Ciudad_-_Medios_Bogota,Medios_Colombianos_por_Ciudad_-_Medios_Cali,Medios_Colombianos_por_Ciudad_-_Medios_Cucuta,Medios_Colombianos_por_Ciudad_-_Medios_Medellin,Medios_Colombianos_por_Ciudad_-_Medios_Nacionales,Medios_Policia_Captura_-_Capturas_a_Venezolanos,Medios_Policia_Captura_-_Todas_las_capturas,Medios_de_Comunicación_-_Alternativos_General,Medios_de_Comunicación_-_Alternativos_Venezolanos,Medios_de_Comunicación_-_Tradicionales_General,Medios_de_Comunicación_-_Tradicionales_Venezolanos
0,2021-02-01 18:24:51.0,#1Feb 🇻🇪🇺🇸 Luego que Joe Biden asumiera la pre...,#1Feb 🇻🇪🇺🇸 Luego que Joe Biden asumiera la pre...,twitter.com,neutral,COL.Bogota.Bogota,individual,fronteravivanew,Bogota,unknown,"#1feb, #somosfronteraviva",8.4,3647,NaN,NaN,NaN,NaN,post,6000000000,3647,471,0,0,13769,false,2331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-02-01 18:23:56.0,"@NDtitulares Que así sea... Los venezolanos, y...","@NDtitulares Que así sea... Los venezolanos, y...",twitter.com,negative,COL.Bogota.Bogota,individual,HAROLDVACCA,Bogota,male,NaN,8.2,7,NaN,@ndtitulares,NaN,NDtitulares,reply,6000000000,7,25,1,0,3885,false,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-02-01 18:23:46.0,RT @ernestosamperp Las revelaciones conocidas ...,RT @ernestosamperp Las revelaciones conocidas ...,twitter.com,neutral,COL.Antioquia.Medellin,individual,gabocano78,Medellin,male,NaN,0,470,NaN,"@ernestosamperp, @nicolasmaduro",NaN,ernestosamperp,share,6000000000,470,985,0,0,52247,false,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-02-01 18:22:16.0,RT @ernestosamperp Las revelaciones conocidas ...,RT @ernestosamperp Las revelaciones conocidas ...,twitter.com,neutral,COL.Antioquia.Medellin,individual,yotagri1895,Medellin,male,NaN,0,120,NaN,"@ernestosamperp, @nicolasmaduro",NaN,ernestosamperp,share,6000000000,120,212,0,0,19852,false,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-02-01 18:21:34.0,Esta también es tu casa • Comunidades de acogi...,Esta también es tu casa • Comunidades de acogi...,twitter.com,neutral,COL.Bogota.Bogota,individual,ficonpaz,Bogota,unknown,"#estatambienestucasamedellince, #sembradoresde...",0,16,NaN,NaN,NaN,ficonpaz,reply,6000000000,16,13,0,0,927,false,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Limpieza con regular expressions
#Eliminamos todos los @Usuario (i.e. @ClaudiaLopez)
df['Clean_Text']=df['Snippet'].replace('@\\w+', '', regex=True)
#Eliminamos todos los hashtags (i.e. #ClaudiaLopezXenofoba)
df['Clean_Text']=df['Clean_Text'].replace('#\\w+', '', regex=True)

df['Clean_Text']=df['Clean_Text'].replace(r"http\S+", '', regex=True)
#Eliminamos todos los signos de puntuación
df['Clean_Text']=df['Clean_Text'].replace(r'[^\w\s]+', '', regex=True)
#Remplazamos los acentos de las palabras y los emojis
df['Clean_Text']=df['Clean_Text'].str.normalize('NFKD')\
        .str.encode('ascii', errors='ignore')\
        .str.decode('utf-8')
#Eliminamos los dobles espacios 
df['Clean_Text']=df['Clean_Text'].replace('\s+', ' ', regex=True).str.strip()

df['Clean_Text'] = df['Clean_Text'].replace(r'(.)\1+', r'\1\1', regex=True).str.strip()
# Eliminamos los RTs
df = df[~df['Clean_Text'].str.startswith('RT')]

df

In [ ]:
#[print(str(i)+". "+msg) for (i, msg) in zip(range(len(df['Clean_Text'])), df['Clean_Text'])]

In [98]:
url = "https://github.com/terrok9/BX/blob/main/Base/medios.txt?raw=true" 
medios = requests.get(url).text

for exp in ('\r', '\n', '\t'):
  medios=medios.replace(exp, ' ') 
medios=medios.rsplit(' OR')
medios=[medio.strip() for medio in medios]

text=[]
for medio in medios:
  if medio.find('.')==-1:
    medio='@'+medio
  text.append(medio)
medios=text

del text

medios[0:10]

['@RevistaSemana',
 '@larepublica_co',
 '@Portafolioco',
 '@RevistaDinero',
 '@NoticiasCaracol',
 '@NoticiasRCN',
 '@Citytv',
 '@InstitucionalTV',
 '@Canal1Colombia',
 '@SenalColombia']

In [ ]:
# Vamos a guardar la lista de autores. Primero policia, luego medios tradicionales y después, alternos
medios_policia = ["@PoliciaBogota", "@region6policia", "@region7policia", "@Region8Policia", "@Region2Policia", "@Region3Policia", "@Region1Policia",
                    "@Region4Policia", "@Region5Policia", "@PoliciaNarino", "@DevalPolicia", "@PoliciaMeta", "@PoliciaMedellin", "@PoliciaBoyaca",
                    "@PoliciaMzales", "@PoliciaVillavo", "@PoliciaBmanga", "@PoliciaCali", "@PoliciaDEMAG", "@PoliciaCmarca", "@Policiantioquia",
                    "@PoliciaCucuta", "@PoliciaQuindio", "@Policia_Caldas", "@SucrePolicia", "@PoliciaVichada", "@PoliciaGuainia",
                    "@PoliciaArauca", "@PoliciaCasanare", "@GuajiraPolicia", "@PoliciaVaupes", "@PoliciaChoco", "@PoliciaDepuy",
                    "@PoliciaUraba", "@PoliciAtlantico", "@PoliciaDECOR", "@PoliciaNteSder", "@HuilaPolicia", "@PoliciaBolivar", "@PoliciaCaqueta",
                    "@PoliciaDeris", "@PoliciaDeTolima", "@PoliciaDEMAM", "@PoliciaMTunja", "@PoliciaMonteria", "@PoliciAmazonas", "@PoliciaDeCesar",
                    "@PoliciaSanyProv", "@PoliciaPasto", "@PoliciaStander", "@PoliciaStaMarta", "@PoliciaPereira", "@PoliciaIbague", "@PoliciaPopayan", 
                    "@PoliciaCtagena", "@PoliciaColombia"]
pattern='|'.join(medios_policia)

In [ ]:
df.loc[df['Mentioned_Authors'].str.contains(pattern, na=False)]['Snippet']

458      @PoliciaMedellin @Guardianes_Ant @metrodemedel...
37807    @PoliciaMedellin @metrodemedellin @FiscaliaCol...
76280    @PoliciaMedellin @metrodemedellin @minuto30com...
80642    @PoliciaMedellin @AlcSabaneta @PoliciaColombia...
Name: Snippet, dtype: object

In [ ]:
df.iloc[80642, 2]

'@PoliciaMedellin @AlcSabaneta @PoliciaColombia @region6policia @minuto30com @QHuboMedallo Jajajajjajaja Que Positivo Tan Hptaaaaa!!! Jajajajajajaj Y Venezolanos Roben Y Atraquen!!'

In [ ]:
temp=['D_C_Bogota OR',
 'Agencia_API OR',
 'ArgumentosyOpi OR',
 'lasillavacia OR',
 'noticucuta OR',
 '2minutosco OR',
 'Elsonajeroweb OR',
 'siaradio OR',
 'noticierolaruta OR',
 'AlertaPaisa OR',
 'orientesetuguia OR',
 'acuariotv OR',
 'cncnoticias OR',
 'Informativoan OR',
 'MiPutumayo OR',
 'zonacero OR',
 'AtlanticoEmi OR',
 'primeronoticiaa1 OR',
 'primeronoticiaa OR',
 'CVnoticiastv OR',
 'LaLibertadCo OR',
 'CanalTropicaltv OR',
 'elambito OR',
 'NorteSeguro2018 OR',
 'areacucuta OR',
 'CucutaDenuncia OR',
 'WALLTHER_VE OR',
 'noticucuta OR',
 'NorteNoticiasCu OR',
 'EnterateCali OR',
 'norte_periodico OR',
 'canalguacari OR',
 'TwiterosCali OR',
 'YaCelacanto OR',
 'HablaManizales OR',
 'tintiando OR',
 'NCTNOTICIAS OR',
 '1anoticiass OR',
 'OsBoyaca OR',
 'aldianoticiasco OR',
 'LaIndependenci1 OR',
 'Prensalibcasan OR',
 'elconucoacacias OR',
 'EIExpedienteCol OR',
 'hablalonoticias OR',
 '7NNoticias OR',
 'ElPregonar OR',
 '45SegundosCom OR',
 'AlPuntoCol OR',
 'ColmundoRadio OR',
 'ElVenezolanoCo OR',
 'ExtraColombia OR',
 'HSBnoticias OR',
 'rodrigoprensa OR',
 'Yariguies1027 OR',
 'BLUSantanderes OR',
 'Villavoalreves OR',
 'Region_365 OR',
 'ElInformador_SM OR',
 'Seguimiento OR',
 'elrotativoc OR',
 'vernoticiassan1 OR',
 'DiarioDelNorte OR',
 'NoticiasGuajira OR',
 'laregionalnet OR',
 'elpaisvallenato OR',
 'Soyquintopoder OR',
 'PrensaLlanera OR',
 'croniquindio OR',
 'agenciaNDS OR',
 'ASAKERW OR',
 'transmisora720 OR',
 'pvirtualcom OR',
 'SuperPopayan OR',
 'HoyDiarioMag OR',
 'CHICACERETE OR',
 'el_regionales OR',
 'NoticiasBQ OR',
 'elhomenoticias OR',
 'HSBnoticias OR',
 'HablemeC OR',
 'elhomenoticias OR',
 'confidencialcol OR',
 'AlertaTolima OR',
 'QHUBOTOLIMA OR',
 'hesnoticia OR',
 'norte_periodico OR',
 'RioNoticiasCo OR',
 'CHICACERETE OR',
 'Coneolavozdelpu OR',
 'lalenguacaribe OR',
 'diarioriente OR',
 'noticiasyabq OR',
 'laotraverdadBAQ OR',
 'noticoopercom OR',
 'noticiasyabq OR',
 'latertuliakilla OR',
 'diarioelfrente OR',
 'TvcucutaC OR',
 'HoyNoticias2 OR',
 'lareporteria OR',
 'CasanareonlineC OR',
 'LATALPAZ OR',
 'Cambio24Oficial OR',
 'LaJornadaco OR',
 'RadioMagdalena OR',
 'EventosStaMta OR',
 'LaGuajira_ OR',
 'hernandomontes6 OR',
 'TuValledupar OR',
 'croniquindio OR',
 'SomosAgroNotas OR',
 'SASComCreativa OR',
 'CostaNoticiasTV OR',
 'duvanedo OR',
 'AlInformativo OR',
 'NPIndependiente OR',
 'kesedice OR',
 'atntelevision OR',
 'anticorrupto26_ OR',
 'FronteraInforma OR',
 'Horadelaverdad OR',
 'SantamartaAD OR',
 'LGCundinamarca OR',
 'Caracol1170AM OR',
 'lavibrante OR',
 'Newsradio7arauc OR',
 'ImpactoMundo_ OR',
 'ElkanalTunja OR',
 'fronteravivanew OR',
 'momento24co OR',
 'AnalisisUrbano OR',
 'SeguridadCucuta OR',
 'CucutaSeguridad OR',
 'FunemeTito OR',
 'El_NuevoOriente OR',
 'radiosantafentc OR',
 'kienyke OR',
 'periodispublico OR',
 'diariodelsur.com.co OR',
 'atlinnovacion.com OR',
 'diariolalibertad.com OR',
 'elcolombiano.com OR',
 'extra.com.co OR',
 'diariodelcauca.com.co OR',
 'hoydiariodelmagdalena.com.co OR',
 'diariodelhuila.com OR',
 'costanoticias.com OR',
 'areacucuta.com OR',
 'confidencialcolombia.com OR',
 'radiosantafe.com']

alternativos=[]
for medio in temp:
  alternativo=medio.replace(' OR', '')
  alternativos.append(alternativo)


In [ ]:
mainstream=['@RevistaSemana',
 '@larepublica_co',
 '@Portafolioco',
 '@RevistaDinero',
 '@NoticiasCaracol',
 '@NoticiasRCN',
 '@Citytv',
 '@InstitucionalTV',
 '@Canal1Colombia',
 '@SenalColombia',
 '@CABLENOTICIAS',
 '@CanalCapital',
 '@CanalTreceCo',
 '@NoticiasUno',
 '@NTN24',
 '@lafm',
 '@BluRadioCo',
 '@dinersrevista',
 '@colombiareports',
 '@mundoinstante',
 '@pulzo',
 '@PublimetroCol',
 '@RedPrensaVerde',
 'semana.com',
 'larepublica.co',
 'portafolio.co',
 'dinero.com',
 'noticias.caracoltv.com',
 'noticias.canalrcn.com',
 'citytv.com.co',
 'canalinstitucional.tv',
 'noticias.canal1.com.co',
 'senalcolombia.tv',
 'cablenoticias.tv',
 'canalcapital.gov.co',
 'canaltrece.com.co',
 'noticiasuno.com',
 'ntn24.com',
 'lafm.com.co',
 'bluradio.com',
 'revistadiners.com.co',
 'colombiareports.com',
 'elmundoalinstante.com',
 'pulzo.com',
 'publimetro.co',
 'redprensaverde.org',
 '@elheraldoco',
 '@LaLibertadCo',
 '@CanalTelecaribe',
 '@QhuboQuilla',
 '@adnbarranquilla',
 'diariolalibertad.com',
 'telecaribe.co',
 'qhubo.com',
 'diarioadn.co',
 '@ELTIEMPO',
 '@elespectador',
 '@ElNuevoSiglo',
 '@malpensante',
 '@RevistaLNE',
 '@citypaperbogota',
 '@BogotaPost',
 '@QhuboBogota',
 '@adncolombia',
 '@Gatopardocom',
 'elespectador.com',
 'elnuevosiglo.com.co',
 'elmalpensante.com',
 'lanotaeconomica.com.co',
 'thecitypaperbogota.com',
 'thebogotapost.com',
 'qhubobogota.com',
 'diarioadn.co',
 'gatopardo.com',
 '@Diarioccidente',
 '@elpaiscali',
 '@TelepacificoTV',
 '@Noti90Minutos',
 '@Redsonoraradio',
 '@QhuboCali',
 '@adncali',
 'elpais.com.co',
 'telepacifico.com',
 '90minutos.co',
 'redsonoraradio.net',
 'qhubocali.com',
 'diarioadn.co',
 '@laopinioncucuta',
 '@QhuboCucuta',
 '@TvcucutaC',
 '@noticucuta',
 '@TUKANAL',
 '@CucutaEsNoticia',
 '@LaVozDelNorteAM',
 '@CaracolCucuta',
 'qhubo.com',
 'tvcucuta.com',
 'noticucuta3.blogspot.com',
 'tukanal.tv',
 'cucutaesnoticia.com',
 '@elcolombiano',
 '@Teleantioquia',
 '@Telemedellin',
 '@minuto30com',
 '@QhuboMedallo',
 '@ADNMedellin',
 '@hora13noticias',
 '@Centropolis_med',
 '@elpalpitar',
 '@ActualidadPol',
 '@DespiertaAnt',
 '@vivirpoblado',
 'teleantioquia.co',
 'telemedellin.tv',
 'minuto30.com',
 'qhubo.com',
 'diarioadn.co',
 'h13n.com',
 'centropolismedellin.com',
 'elpalpitar.com',
 'actualidad.com.co',
 'despiertaantioquia.com',
 'vivirenelpoblado.com']

In [ ]:
mencionables=[]
mencionables.extend(alternativos)
mencionables.extend(mainstream)

for cuenta in range(len(mencionables)):
  if mencionables[cuenta][0]!='@':
    mencionables[cuenta]='@'+mencionables[cuenta]

pattern='|'.join(mencionables)



In [ ]:
len(df.loc[df['Mentioned_Authors'].str.contains(pattern, na=False)]['Snippet'])

1213